<a href="https://colab.research.google.com/github/Bibhash123/Project-Primary-Quantization/blob/main/experiments/primary_quantization_two_head_densenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
COLAB = False
if COLAB:
    DATA_DIR = "/content/Data"
else:
    DATA_DIR = "../input/tondidataset/"

In [ ]:
if COLAB:
    from google.colab import files
    _ = files.upload()
    !mkdir ~/.kaggle/
    !cp kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json
    !pip install -q kaggle
    !kaggle datasets download -d "bibhash123/tondidataset"
    !unzip -q tondidataset.zip -d "/content/Data/"
    !rm -r tondidataset.zip

In [ ]:
import os
import gc
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from functools import partial

In [ ]:
train_files = pd.read_csv(os.path.join(DATA_DIR, "mini_train.csv"),sep=";",header=None,
                         names=['idx', 'filenames', 'quality1', 'quality2', 'software', 'labels',
                                'shift_r', 'shift_c','base_image']
                         )
trn,val = train_test_split(train_files,test_size=0.1)

In [ ]:
print("Train Shape: ", trn.shape)
print("Validation Shape: ", val.shape)

Train Shape:  (97200, 9)
Validation Shape:  (10800, 9)


In [ ]:
def preprocess_input(im_file, target_size, scale=255.):
    """ 
        Read image and (eventually) scale data
        Arguments:
            im_file     : input image file
            target_size : output size of the image (height, width)
            scale       : pixel scaling value
        Returns: The image
    """
    file_bytes = tf.io.read_file(im_file)
    img = tf.image.decode_png(file_bytes, channels = 0)
    # Normalize and Resize
#     if img.shape != target_size:
#         img = tf.image.resize(img, target_size)
    img = tf.cast(img, tf.float32)
    return img
  
def string2Q(s, size=(8, 8), flatten=True):
    """ Converts a comma separated string to a matrix.
        Keyword arguments:
        sq : input string
        size : output matrix size
    """
    if flatten:
        return tf.strings.to_number(tf.strings.split(s,','),out_type=tf.int32)
    else:
        return tf.reshape(tf.strings.to_number(tf.strings.split(s,','),out_type=tf.int32),size)
        
def get_label(im_label):
    return string2Q(im_label)[:15]

def getQFRange(qf1):
    return K.stack([qf1-5,qf1,qf1+5],axis=0)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
QFS = [60,65,70,75,80,85,90,98]

def build_decoder(is_labelled):
    def if_labelled(path,label, target_size):
        image = preprocess_input(path,target_size)
        out_img = K.concatenate([tf.expand_dims(255.0*tf.image.adjust_jpeg_quality(image/255.0,qf),axis=0) for qf in QFS],axis=0)
        label = get_label(label)
        # qf = getQFRange(qf1)
        return (image, out_img), label
  
    def not_labelled(path,target_size):
        image = preprocess_input(path,target_size)
        out_img = K.concatenate([tf.expand_dims(255.0*tf.image.adjust_jpeg_quality(image/255.0,qf),axis=0) for qf in QFS],axis=0)
        return (image, out_img)    

    return if_labelled if is_labelled else not_labelled


def create_dataset(df, batch_size = 32, is_labelled = False, repeat = False, shuffle = False, batch=False, cache=False):
    decode_fn = build_decoder(is_labelled)
    df['filenames'] = df['filenames'].apply(lambda x: "/".join(x.split('/')[-3:]))
    df['filenames'] = df['filenames'].apply(lambda x: os.path.join(DATA_DIR,x))
    
    # Create Dataset
    if is_labelled:
        dataset = tf.data.Dataset.from_tensor_slices((df['filenames'].values,df["labels"].values))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df['filenames'].values))

    dataset = dataset.map(partial(decode_fn,target_size=(64,64)), num_parallel_calls = AUTOTUNE)
    dataset = dataset.cache("") if cache else dataset
    dataset = dataset.repeat() if repeat else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration = True) if shuffle else dataset
    dataset = dataset.batch(batch_size,drop_remainder=True) if batch else dataset
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def getX(X,Y):
    return X
def getY(X,Y):
    return Y

In [ ]:
train_set = create_dataset(trn, batch_size = 32, is_labelled = True, repeat = True, 
                          shuffle = True, batch=True,cache=False)
val_set = create_dataset(val, batch_size = 32, is_labelled = True, repeat = False, 
                          shuffle = False, batch=True,cache=False)
Y_val = val_set.map(getY)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
2022-03-07 14:14:09.481842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 14:14:09.575238: I tensorflow/stream_executor/cuda/cuda_gpu_exe

In [ ]:
class CrossAttention(L.Layer):
    def __init__(self,input_dim, hidden_dim, output_dim):
        super(CrossAttention,self).__init__()
        self.w_k = self.add_weight(name='key', shape = (input_dim,hidden_dim), initializer="random_normal", trainable=True)
        self.w_q = self.add_weight(name='query', shape = (input_dim,hidden_dim), initializer="random_normal", trainable=True)
        self.w_v = self.add_weight(name='value', shape = (input_dim,output_dim), initializer="random_normal", trainable=True)

    def call(self,X1,X2):
        dims = len(QFS)
        key = tf.expand_dims(tf.matmul(X1,self.w_k),axis=1)
        query = K.concatenate([tf.expand_dims(tf.matmul(X2[:,i,:],self.w_q),axis=2) for i in range(dims)],axis=2)
        value = K.concatenate([tf.expand_dims(tf.matmul(X2[:,i,:],self.w_v),axis=2) for i in range(dims)],axis=2)
        score = tf.matmul(tf.nn.softmax(tf.matmul(key,query),axis=-1),tf.transpose(value,perm=[0,2,1]))
        return score[:,0,:]

In [ ]:
class DCT(L.Layer):
    def __init__(self):
        super(DCT,self).__init__()
        filters = []
        for i in range(8):
            for j in range(8):
                f = []
                k = 0.25*self.C(i)*self.C(j)
                for x in range(8):
                    f.append([k*np.cos(((2*x+1)*i*np.pi)/16) * np.cos(((2*y+1)*j*np.pi)/16) for y in range(8)])
                filters.append(K.constant(np.stack(f,axis=0)))
                
        self.conv = L.Conv2D(64,(8,8),strides = 8,activation=None,use_bias=False,padding='same')
        self.conv.build((64,64,1))
        weights = self.conv.get_weights()
        for i in range(64):
            weights[0][:,:,0,i] = filters[i]
        
        self.conv.set_weights(weights)
        self.trainable = False
        self.conv.trainable = False
        self.filters = filters
        
    def C(self,x):
        if x==0:
            return 1/(2**0.5)
        elif x>0:
            return 1
    
    def call(self,x):
        h = self.conv(x)
        return h
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[1]//8,input_shape[2]//8,64)

from PIL import Image
dct = DCT()
inp = np.expand_dims(np.expand_dims(np.asarray(Image.open('../input/tondidataset/Train/60-90/00000024_rcb192c6at.TIF.png')),axis=-1),axis=0)
inp = K.constant(inp.astype(float))
dct_out = dct(inp)
dct_out.shape

2022-03-07 14:14:12.980843: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


TensorShape([1, 8, 8, 64])

In [ ]:
class Histogram(L.Layer):
    def __init__(self, b0,b1, num_bins, gamma):
#         self.b0 = self.add_weight(name='b0', shape=[1], initializer="random_normal", trainable=True)
#         self.b1 = self.add_weight(name='b1', shape=[1], initializer="random_normal", trainable=True)
        super(Histogram,self).__init__()
        self.b_ = np.linspace(b0,b1,num_bins)
        self.gamma = gamma
        self.avg_pool = L.AveragePooling2D(pool_size = (8,8), strides = 8)
        self.diff_filter = np.array([[1,-1]]).reshape((2,1))
        self.differentiator = L.Conv2D(1,(2,1),strides=1,activation=None, padding = 'same', use_bias = False)
        self.differentiator.build((num_bins,64,1))
        weights = self.differentiator.get_weights()
        weights[0][:,:,0,0] = K.constant(self.diff_filter)
        self.differentiator.set_weights(weights)
        self.trainable = False
        self.differentiator.trainable = False
    
    def call(self,X):
        h = L.Reshape((64,64))(X)
        matrix = [64**2 * K.mean(tf.nn.sigmoid(self.gamma*(h-b)),axis=1) for b in self.b_]
#         matrix = [[64**2 * self.avg_pool(tf.expand_dims(tf.nn.sigmoid(self.gamma*(X[:,:,:,freq_pair]- b)),axis=-1))[:,0,0,0] 
#                    for freq_pair in range(64)] for b in self.b_]
#         matrix = tf.expand_dims(tf.transpose(K.stack(matrix,axis=0), perm=[2,0,1]),axis=-1)
        matrix = tf.expand_dims(tf.transpose(K.stack(matrix,axis=0), perm=[1,0,2]),axis=-1)

        return self.differentiator(matrix)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0],len(self.b_),64,1)
    
hist = Histogram(-200,200,20,1e5)
hist_out = hist(dct_out)
hist_out.shape

TensorShape([1, 20, 64, 1])

In [ ]:
class ConvModel(tf.keras.models.Model):
    def __init__(self,num_classes=10):
        super(ConvModel,self).__init__()
        self.n_classes = num_classes
        self.block1 = L.Conv2D(16,(3,3),strides=1,input_shape=(None,20,64,1))

        self.block2 = tf.keras.models.Sequential([
                                                  L.Conv2D(16,(3,3),strides=1),
                                                  L.BatchNormalization(),
                                                  L.Activation('relu')
        ])
        self.block3 = tf.keras.models.Sequential([
                                                 L.Conv2D(32,(3,3),strides=1),
                                                 L.BatchNormalization(),
                                                 L.Activation('relu')
        ])
        self.block4 = tf.keras.models.Sequential([
                                                 L.Conv2D(64,(3,3),strides=1),
                                                 L.BatchNormalization(),
                                                 L.Activation('relu')
        ])
        self.block5 = tf.keras.models.Sequential([
                                                 L.Conv2D(128,(1,1),strides=1),
                                                 L.BatchNormalization(),
                                                 L.Activation('relu'),
                                                 L.AveragePooling2D(pool_size=(2,2),strides=2)
        ])
        
        self.flatten = L.Flatten()
        self.fc1 = L.Dense(200, activation='relu')
        self.fc2 = L.Dense(100, activation='relu')
#         self.fc3 = L.Dense(num_classes,activation="linear")

    def call(self,inputs,**kwargs):
        out = self.block1(inputs)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)

        out = self.flatten(out)

        out = self.fc1(out)
        out = self.fc2(out)
#         out = self.fc3(out)
        return out

    def compute_output_shape(self,input_shape):
        return (input_shape[0],100)

In [ ]:
from densenet import DenseNet, ConvConst
K.clear_session()
with tf.device('/GPU:0'):
#     err_model = ConvModel(num_classes = 15)
    err_model,_ = DenseNet(input_shape = (20,64,1), nb_classes = 15, depth=18)
    err_model = tf.keras.Model(inputs=err_model.input, outputs = err_model.layers[-2].output)
#     err_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights=None, input_shape=(64,64,1))
#     img_model = ConvModel(num_classes = 15)
    img_model,_ = DenseNet(input_shape = (20,64,1), nb_classes = 15, depth=18)
    img_model = tf.keras.Model(inputs=img_model.input, outputs = img_model.layers[-2].output)
#     img_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights=None, input_shape=(64,64,1))

    inp1 = L.Input(shape=(64,64,1))
    inp2 = L.Input(shape=(len(QFS),64,64,1))
    h1 = DCT()(inp1)
    h1 = Histogram(-200,200,20,1e5)(h1)
    h1 = img_model(h1)
    
    h2 = L.TimeDistributed(DCT())(inp2)
    h2 = L.TimeDistributed(Histogram(-200,200,20,1e5))(h2)
    h2 = L.TimeDistributed(err_model)(h2)
    h2 = L.TimeDistributed(L.Flatten())(h2)
    h2 = L.TimeDistributed(L.Dense(64,activation='relu'))(h2)
    h1 = L.Flatten()(h1)
    h1 = L.Dense(64,activation='relu')(h1)
    
    h = CrossAttention(64,32,64)(h1,h2)
    h = L.Dense(15,activation='linear')(h)
    
    model = tf.keras.Model(inputs = [inp1,inp2],
                         outputs = h)
    
#     schedule = tf.keras.optimizers.schedules.ExponentialDecay(1e-4,
#                                                               decay_steps=9000,
#                                                               decay_rate=0.96,
#                                                               staircase=True)
    
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-5, clipnorm = 1.0)
    model.compile(loss = tf.keras.losses.LogCosh(), optimizer = opt)
    model.summary()

Creating DenseNet
#############################################
Dense blocks: 3
Layers per dense block: [4, 4, 4]
#############################################
Creating DenseNet
#############################################
Dense blocks: 3
Layers per dense block: [4, 4, 4]
#############################################
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 8, 64, 64, 1 0                                            
__________________________________________________________________________________________________
dct (DCT)                       (None, 8, 8, 64)     4096        inpu

In [ ]:
class Logger(tf.keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs={}):
        print(" Learning Rate: {}".format(self.model.optimizer._decayed_lr(tf.float32).numpy()))

In [ ]:
class ConstWeight(tf.keras.callbacks.Callback):
    def on_batch_end(self,epoch,logs={}):
        weights = self.model.layers[2].layers[0].get_weights()
        weights[0][2,2,0,:] = np.zeros((12,))
        for i in range(weights[0].shape[-1]):
            t = weights[0][:,:,0,i]
            nom = -1*np.sum(t)
            weights[0][:,:,:,i] = weights[0][:,:,:,i]/nom
        weights[0][2,2,0,:] = np.ones((12,))  
        self.model.layers[2].layers[0].set_weights(weights)

        weights = self.model.layers[3].model.layers[0].get_weights()
        weights[0][2,2,0,:] = np.zeros((12,))
        for i in range(weights[0].shape[-1]):
            t = weights[0][:,:,0,i]
            nom = -1*np.sum(t)
            weights[0][:,:,:,i] = weights[0][:,:,:,i]/nom
        weights[0][2,2,0,:] = np.ones((12,))  
        self.model.layers[3].model.layers[0].set_weights(weights)

In [ ]:
ckpt = tf.keras.callbacks.ModelCheckpoint('model.hdf5', monitor = 'val_loss', mode='min',
                                          save_best_only = True, save_weights_only = True)
es = tf.keras.callbacks.EarlyStopping(patience = 7, monitor = 'val_loss', mode='min',
                                      restore_best_weights=True)
logger = Logger()
# const = ConstWeight()
try:
    # model.load_weights('last_trained.hdf5')
    model.fit(train_set,
            epochs = 60,
            steps_per_epoch = (trn.shape[0]//32),
            validation_data = val_set,
            callbacks = [ckpt,es],
            initial_epoch=0
            )
except KeyboardInterrupt:
    print("\n[INFO] Interrupted Training")
    model.save_weights('last_trained.hdf5')
print('[INFO] Obtaining Predictions')
model.load_weights('model.hdf5')
pred = model.predict(val_set)

Epoch 1/60


2022-03-07 14:14:28.096594: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


3037/3037 [==============================] - 409s 130ms/step - loss: 3.7995 - val_loss: 3.0000
Epoch 2/60
3037/3037 [==============================] - 393s 129ms/step - loss: 2.8721 - val_loss: 2.7316
Epoch 3/60
3037/3037 [==============================] - 389s 128ms/step - loss: 2.6275 - val_loss: 2.8370
Epoch 4/60
3037/3037 [==============================] - 388s 128ms/step - loss: 2.5456 - val_loss: 2.5757
Epoch 5/60
3037/3037 [==============================] - 395s 130ms/step - loss: 2.4983 - val_loss: 2.7911
Epoch 6/60
3037/3037 [==============================] - 398s 131ms/step - loss: 2.4492 - val_loss: 2.6576
Epoch 7/60
3037/3037 [==============================] - 397s 131ms/step - loss: 2.4149 - val_loss: 2.9895
Epoch 8/60
3037/3037 [==============================] - 392s 129ms/step - loss: 2.3772 - val_loss: 2.5285
Epoch 9/60
3037/3037 [==============================] - 386s 127ms/step - loss: 2.3398 - val_loss: 3.4206
Epoch 10/60
3037/3037 [==============================] - 

In [ ]:
a = list(train_set.take(1).as_numpy_iterator())[0]

In [ ]:
print(a[0][0].shape)
print(a[0][1].shape)
print(a[1].shape)

(32, 64, 64, 1)
(32, 8, 64, 64, 1)
(32, 15)


In [ ]:
y = np.array(list(Y_val.unbatch().as_numpy_iterator()))
print("Shape Y: ",y.shape)
print("Accuracy = {:.4f}".format(np.sum(np.round(pred)==y)/(15*y.shape[0])))
print("MSE = {:.4f}".format(np.sum(np.square(y - pred)) / (y.shape[0]*15)))